# SignXAI2 TensorFlow Advanced Tutorial - Image Classification

This advanced tutorial demonstrates sophisticated analysis techniques using SignXAI2 with TensorFlow, including class-specific explanations and positive/negative contribution separation.

## Prerequisites

Complete the basic TensorFlow tutorial first, and ensure you have the required data and model setup.

⚠️ **Data Requirements**: This tutorial requires example data from the GitHub repository.

## Setup and Basic Model Loading

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from signxai import explain, list_methods
from signxai.utils.utils import normalize_heatmap
import urllib.request

# Download an example image
url = "https://farm1.staticflickr.com/148/414245159_7549a49046_z.jpg"
urllib.request.urlretrieve(url, "dog.jpg")

# Load the pre-trained model
model = VGG16(weights='imagenet')

# Remove softmax layer (critical for explanations)
model.layers[-1].activation = None

# Load and preprocess the image
img_path = "dog.jpg"
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Make prediction
preds = model.predict(x)
top_pred_idx = np.argmax(preds[0])
print(f"Predicted class: {decode_predictions(preds, top=1)[0][0][1]}")

## Advanced Analysis

Let's compare class-specific explanations:

In [ ]:
# Get top 3 predicted classes
top_classes = np.argsort(preds[0])[-3:][::-1]
class_names = [decode_predictions(preds, top=3)[0][i][1] for i in range(3)]

# Calculate explanations for each class using a complex method with parameter chaining
# This demonstrates: gradient (base) + x_input (multiply by input) + x_sign (apply sign) + mu_neg_0_5 (parameter)
class_explanations = {}
for idx in top_classes:
    class_explanations[idx] = explain(
        model=model,
        x=x,
        method_name='gradient_x_input_x_sign_mu_neg_0_5',
        target_class=idx
    )

# Visualize
fig, axs = plt.subplots(1, 4, figsize=(20, 5))

# Original image
axs[0].imshow(img)
axs[0].set_title('Original Image', fontsize=14)
axs[0].axis('off')

# Class-specific explanations
for i, (idx, name) in enumerate(zip(top_classes, class_names)):
    axs[i+1].imshow(normalize_heatmap(class_explanations[idx][0]), cmap='seismic', clim=(-1, 1))
    axs[i+1].set_title(f'Class: {name}', fontsize=14)
    axs[i+1].axis('off')

plt.tight_layout()
plt.show()

## Positive and Negative Contribution Separation

We can also highlight the positive and negative contributions separately:

In [ ]:
# Choose a complex method with parameter chaining and generate explanation
# This showcases: gradient (base) + x_input (multiply by input) + x_sign (apply sign) + mu_neg_0_5 (parameter)
method = 'gradient_x_input_x_sign_mu_neg_0_5'
explanation = explain(
    model=model,
    x=x,
    method_name=method,
    target_class=top_pred_idx
)[0]

# Separate positive and negative contributions
pos_expl = np.maximum(0, explanation)
neg_expl = np.minimum(0, explanation)

# Normalize
pos_norm = pos_expl / np.max(pos_expl) if np.max(pos_expl) > 0 else pos_expl
neg_norm = neg_expl / np.min(neg_expl) if np.min(neg_expl) < 0 else neg_expl

# Visualize
fig, axs = plt.subplots(1, 4, figsize=(20, 5))

# Original image
axs[0].imshow(img)
axs[0].set_title('Original Image', fontsize=14)
axs[0].axis('off')

# Combined explanation
axs[1].imshow(normalize_heatmap(explanation), cmap='seismic', clim=(-1, 1))
axs[1].set_title(f'{method} - Combined', fontsize=14)
axs[1].axis('off')

# Positive contributions
axs[2].imshow(pos_norm, cmap='Reds')
axs[2].set_title('Positive Contributions', fontsize=14)
axs[2].axis('off')

# Negative contributions
axs[3].imshow(-neg_norm, cmap='Blues')
axs[3].set_title('Negative Contributions', fontsize=14)
axs[3].axis('off')

plt.tight_layout()
plt.show()

## Summary

In this advanced tutorial, we've demonstrated how to:

1. **Class-specific Analysis**: Generate explanations for different predicted classes to understand what features the model associates with each class
2. **Contribution Separation**: Separate positive and negative contributions to better understand how different regions support or oppose the prediction
3. **Advanced Visualization**: Create comprehensive visualizations that reveal different aspects of the model's decision-making process

These techniques provide deeper insights into model behavior and can help identify potential biases or areas for model improvement.